In [326]:
import json
from web3 import Web3
from collections import defaultdict

tca = lambda x: w3.toChecksumAddress(x)


class StateChannelBackend:
    
    def __init__(self):    
        with open("../project/build/contracts/StateChannel.json", "r") as f:
            contract_data = json.load(f)
            
        address = tca(contract_data["networks"]["5777"]["address"])
        
        self.contract = w3.eth.contract(address=address, abi=contract_data["abi"])
        self.owner = tca(self.contract.functions.owner().call())
        self.receipts = defaultdict(defaultdict)

    def accepting_new_channels(self) -> bool:
        return self.contract.functions.accepting_new_channels().call()
    
    def _transact_contract(self, method, t_params: dict):
        fs = method.transact(t_params)
        w3.eth.waitForTransactionReceipt(fs)
    
    def set_accepting_new_channels(self, val: bool):
        transact_params = {"from": self.owner}
        self._transact_contract(self.contract.functions.changeAcceptanceStatus(val), transact_params)

    def receiveReceipt(self, receipt):
        r = Receipt(**receipt)
        resp = self.contract.functions.verify_receipt(receipt.allowed_funds, 
                                                      receipt.account, 
                                                      receipt.channel_number, 
                                                      v, r, s).call()
        if not resp:
            return False
        
        self.receipts[receipt.account][receipt.channel_number] = receipt
        return True


In [327]:
st = StateChannelBackend()
st.set_accepting_new_channels(True)
assert (st.accepting_new_channels() == True)
# st.set_accepting_new_channels(False)
# assert (st.accepting_new_channels() == False)

In [329]:
st.owner

'0xd1687a6295283061450ac168f5C03e309BB9CDc4'

In [322]:
w3.eth.getBalance(st.owner) > 9000000000000000000

True

In [323]:
punishment = st.contract.functions.PUNISHMENT().call()
punishment

1000000000000000000

In [328]:
assert (st.accepting_new_channels() == True)
st.contract.functions.createNewChannel(1000000000000000000).transact({"from": st.owner, 
                                                                     "value": 1000000000000000000 + punishment})

HexBytes('0x941a617507e0c7132e2bd86df55507313ab00dee881499a00663f352ffa1e189')

In [306]:
err = "{'message': 'VM Exception while processing transaction: revert', 'code': -32000, 'data': {'stack': 'Error\n    at Function.c.fromResults (/tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/rustbn.js/index.js:3:1)\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/ganache-core/lib/blockchain_double.js:491:1\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/ethereumjs-vm/lib/runBlock.js:7:1\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:4080:1\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:480:1\n    at o (/tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:1030:1)\n    at f (/tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:998:1)\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:966:1\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:4079:1\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/internal/eachOfLimit.js:3:1\n    at l (/tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/internal/eachOfLimit.js:16:1)\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/internal/eachOfLimit.js:28:39\n    at t.default (/tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/eachLimit.js:20:1)\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/internal/slice.js:3:1\n    at g.n.emit (/tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async-eventemitter/lib/AsyncEventEmitter.js:58:1)\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/ethereumjs-vm/lib/runBlock.js:3:1\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:3947:1\n    at o (/tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:1031:1)\n    at f (/tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:998:1)\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:966:1\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:4079:1\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/ethereumjs-vm/lib/runBlock.js:1:1\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/ethereumjs-vm/lib/cache.js:117:1\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:480:1\n    at o (/tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:1030:1)\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:1032:1\n    at Ut (/tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:3245:1)\n    at Object.<anonymous> (/tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:1049:10)\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/ethereumjs-vm/lib/cache.js:113:1\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:966:1\n    at c (/tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:5240:1)\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/ethereumjs-vm/lib/cache.js:107:1\n    at c (/tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:5239:1)\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/ethereumjs-vm/lib/cache.js:105:1', 'name': 'c'}}"

In [330]:
st = StateChannel()
owner = st.owner

In [336]:
acc = "0xE58ea859e7DE7EaB1328A730CB397d9597F5aDC6"

In [345]:
msg = w3.soliditySha3(["uint256", "uint256", "uint256"], [4, int(acc, 16), 4])
signature = w3.eth.sign(acc, msg)

signature = signature.hex()

r = signature[:66]
s = '0x' + signature[66:130]
v = '0x' + signature[130:132]
v = int(v, 16)
v = v if v in [27, 28] else v + 27

r,s,v

st.contract.functions.verify_receipt(4, acc, 4, v, r, s).call()

ValueError: {'message': 'VM Exception while processing transaction: revert', 'code': -32000, 'data': {'stack': 'Error\n    at Function.c.fromResults (/tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/rustbn.js/index.js:3:1)\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/ganache-core/lib/blockchain_double.js:491:1\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/ethereumjs-vm/lib/runBlock.js:7:1\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:4080:1\n    at /tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_modules/async/dist/async.js:480:1\n    at o (/tmp/.mount_ganachrwhBTn/app/resources/app.asar/node_modules/ganache-cli/build/webpack:/ganache/node_module

In [261]:
from collections import namedtuple

In [ ]:
namedtuple()